# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()



   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables


In [4]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab([df.prestige, df.admit], columns="count")


col_0           count
prestige admit       
1.0      0         28
         1         33
2.0      0         95
         1         53
3.0      0         93
         1         28
4.0      0         55
         1         12

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
prestige_dummies = pd.get_dummies(df['prestige'])
print prestige_dummies.head()


   1.0  2.0  3.0  4.0
0  0.0  0.0  1.0  0.0
1  0.0  0.0  1.0  0.0
2  1.0  0.0  0.0  0.0
3  0.0  0.0  0.0  1.0
4  0.0  0.0  0.0  1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [6]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummies.ix[:, 0:4])
print handCalc.head()



   admit    gre   gpa  1.0  2.0  3.0  4.0
0      0  380.0  3.61  0.0  0.0  1.0  0.0
1      1  660.0  3.67  0.0  0.0  1.0  0.0
2      1  800.0  4.00  1.0  0.0  0.0  0.0
3      1  640.0  3.19  0.0  0.0  0.0  1.0
4      0  520.0  2.93  0.0  0.0  0.0  1.0


In [7]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

# do *not* do this to rename columns, reassign like columns = ['col1', 'col2']
# handCalc.columns.values[3]='prestige1'
# handCalc.columns.values[4]='prestige2'
# handCalc.columns.values[5]='prestige3'

print handCalc.columns.tolist()

pd.crosstab(handCalc['admit'], handCalc[1.0], rownames=['admit'])



['admit', 'gre', 'gpa', 1.0, 2.0, 3.0, 4.0]


1.0,0.0,1.0
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college


In [8]:
# Probability = occurrence / total possibilities
# Odds = p / (1 - p)    
probabilityDid = (float(33) / (28 + 33))
oddsDid = probabilityDid / (1 - probabilityDid)

print "probability Did:", probabilityDid
print "odds Did:", oddsDid

probability Did: 0.540983606557
odds Did: 1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [9]:
probabilityDidNot = (float(28) / (28 + 33))
oddsDidNot = probabilityDidNot / (1 - probabilityDidNot)
print "Probability Did Not:", probabilityDidNot
print "Odds Did Not:", oddsDidNot

Probability Did Not: 0.459016393443
Odds Did Not: 0.848484848485


#### 3.3 Calculate the odds ratio

In [10]:
# Odds that an outcome will occurr given a certain exposure/condition 
# compared to the odds of that outcome occurring without that certain exposure/condition

# OR = 1 no association. Exposure/condition does not impact outcome. 
# OR > 1 means the exposure/condition is associated with higher odds of the outcome
# OR < 1 means the exposure/condition is associated with lower odds of the outcome


oddsDid / oddsDidNot


1.389030612244898

#### 3.4 Write this finding in a sentenance: 

Answer: 

Because the OR is greater than 1, it means that there is an association between students who attended a #1 ranked school and being admitted.


#### 3.5 Print the cross tab for prestige_4

In [11]:
pd.crosstab(handCalc['admit'], handCalc[4.0])

4.0,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [12]:
probabilityDidFour = (float(12) / (55 + 12))  # P(getting admitted with a rank 4 school)
oddsDidFour = probabilityDidFour / (1 - probabilityDidFour) # odds of getting admitted with a rank 4 school
print "oddsDidFour:", oddsDidFour


probabilityDidNotFour = (float(55) / (55 + 12))  # P(NOT getting admitted with a rank 4 school)
oddsDidNotFour = probabilityDidNotFour / (1 - probabilityDidNotFour) # odds of not getting admitted with rank4 school
print "oddsDidNotFour:", oddsDidNotFour

rank4OR = oddsDidFour / oddsDidNotFour

print "Getting admitted with a rank 4 school OR:", rank4OR

oddsDidFour: 0.218181818182
oddsDidNotFour: 4.58333333333
Getting admitted with a rank 4 school OR: 0.0476033057851


#### 3.7 Write this finding in a sentence

Answer:  The odds ratio of getting admitted with a rank 4 school is less than 1, which means having a rank 4 school is associated with lower odds of getting admitted. 

## Part 4. Analysis

In [13]:
dummy_ranks = prestige_dummies
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 2.0:])
print data.head()


   admit    gre   gpa  2.0  3.0  4.0
0      0  380.0  3.61  0.0  1.0  0.0
1      1  660.0  3.67  0.0  1.0  0.0
2      1  800.0  4.00  0.0  0.0  0.0
3      1  640.0  3.19  0.0  0.0  1.0
4      0  520.0  2.93  0.0  0.0  1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [14]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols



In [15]:
train_cols = data[['gre', 'gpa', 2.0, 3.0, 4.0]]

In [16]:
train_cols.head()

,gre,gpa,2.0,3.0,4.0
0,380.0,3.61,0.0,1.0,0.0
1,660.0,3.67,0.0,1.0,0.0
2,800.0,4.00,0.0,0.0,0.0
3,640.0,3.19,0.0,0.0,1.0
4,520.0,2.93,0.0,0.0,1.0


#### 4.2 Fit the model

In [17]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
lm.fit(train_cols, data['admit']) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### 4.3 Print the summary results

In [18]:
score = lm.score(train_cols, data['admit'])
print "score:", score

coefficients = lm.coef_
print "coefficients", lm.coef_

score: 0.705289672544
coefficients [[ 0.00178497  0.23229458 -0.60347467 -1.17214957 -1.37729795]]


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [19]:
print lm.coef_
print np.exp(lm.coef_)  # odds ratios of coeffiecients??

[[ 0.00178497  0.23229458 -0.60347467 -1.17214957 -1.37729795]]
[[ 1.00178657  1.26149128  0.546908    0.3097005   0.25225925]]


#### 4.5 Interpret the OR of Prestige_2

Answer:  OR of prestige 2 is 0.546908 which is less than 1 which means that having a rank 2 school is associated with lower odds of getting admitted.

#### 4.6 Interpret the OR of GPA

Answer: OR OF GPA is 1.2614128 which is greater than 1. This means having a higher GPA is associated with greater odds of being admitted.

(How was the GPA calculated if it wasn't binary?)

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [20]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out



In [24]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
print "combos:"
print combos.head()

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]
combos:
       0         1    2    3
0  220.0  2.260000  1.0  1.0
1  220.0  2.260000  2.0  1.0
2  220.0  2.260000  3.0  1.0
3  220.0  2.260000  4.0  1.0
4  220.0  2.453333  1.0  1.0


#### 5.1 Recreate the dummy variables

In [29]:
# recreate the dummy variables
dummies = pd.get_dummies(combos[2])
print dummies.head()

   1.0  2.0  3.0  4.0
0  1.0  0.0  0.0  0.0
1  0.0  1.0  0.0  0.0
2  0.0  0.0  1.0  0.0
3  0.0  0.0  0.0  1.0
4  1.0  0.0  0.0  0.0


In [30]:
# keep only what we need for making predictions
cols_to_keep_combos = combos[[0, 1]]
print cols_to_keep_combos.head()

       0         1
0  220.0  2.260000
1  220.0  2.260000
2  220.0  2.260000
3  220.0  2.260000
4  220.0  2.453333


In [45]:
combos_df = cols_to_keep_combos.join(dummies.ix[:, 2:4])
combos_df.head()

,0.0,1.0,2.0,3.0,4.0
0,220.0,2.260000,0.0,0.0,0.0
1,220.0,2.260000,1.0,0.0,0.0
2,220.0,2.260000,0.0,1.0,0.0
3,220.0,2.260000,0.0,0.0,1.0
4,220.0,2.453333,0.0,0.0,0.0


In [47]:
combos_df.columns = ['gre', 'gpa', 'prest_2', 'prest_3', 'prest_4']
combos_df.head()

,gre,gpa,prest_2,prest_3,prest_4
0,220.0,2.260000,0.0,0.0,0.0
1,220.0,2.260000,1.0,0.0,0.0
2,220.0,2.260000,0.0,1.0,0.0
3,220.0,2.260000,0.0,0.0,1.0
4,220.0,2.453333,0.0,0.0,0.0


#### 5.2 Make predictions on the enumerated dataset

In [50]:
lm.predict(combos_df)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1,

#### 5.3 Interpret findings for the last 4 observations

In [51]:
combos_df.tail()

,gre,gpa,prest_2,prest_3,prest_4
395,800.0,3.806667,0.0,0.0,1.0
396,800.0,4.000000,0.0,0.0,0.0
397,800.0,4.000000,1.0,0.0,0.0
398,800.0,4.000000,0.0,1.0,0.0
399,800.0,4.000000,0.0,0.0,1.0


Answer: Even though all of the last four observations had high GRE and GPA's, only the one with a #1 ranked school got admitted. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.